# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'local', 'met', 'uk_food']


In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [6]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [8]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'The Pines Garden',
 'AddressLine2': 'Beach Road',
 'AddressLine3': 'St Margarets Bay',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Tea Room',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.362402580997,
 'FHRSID': 551803,
 'LocalAuthorityBusinessID': 'PI/000070948',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DZ',
 'RatingDate': '2021-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('646932df32d234eb37da90a6'),
 'geocode': {'latitude': '51.148133', 'longitude': '1.383298'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/551803',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate':

In [10]:
# assign the collection to a variable
# establishments = db['establishments'] # method 1
establishments = db.establishments # method 2

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [18]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
}

In [19]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

In [20]:
# Check that the new restaurant was inserted
pprint(establishments.find_one({"BusinessName":"Penang Flavours"}))

{'BusinessName': 'Penang Flavours', '_id': ObjectId('64694c766cd614e143e930f5')}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [45]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
fields = {
    "BusinessTypeID": 1,
    "BusinessType": 1,
    "_id": 0
}

rcc_id = establishments.find_one(query, fields)
pprint(rcc_id)


{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [41]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one({"BusinessName": "Penang Flavours"}, {"$set": rcc_id})

In [46]:
# Confirm that the new restaurant was updated
establishments.find_one({"BusinessName": "Penang Flavours"})

{'_id': ObjectId('64694c766cd614e143e930f5'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1}

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [48]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_query = {"LocalAuthorityName": "Dover"}
pprint(establishments.count_documents(dover_query))

994


In [60]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many(dover_query)

In [61]:
# Check if any remaining documents include Dover
dover_reg_query = {"LocalAuthorityName": {"$regex": "Dover"}}
pprint(establishments.count_documents(dover_reg_query))

0


In [63]:
# Check that other documents remain with 'find_one'
pprint(establishments.find_one())

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('646932df32d234eb37da938d'),
 'geocode': {'latitude': '51.086058', 'longitude': '1.196408'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          '

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [87]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many(
    {},
    [
        {
            "$set": {
                "geocode.longitude": {"$toDouble": "$geocode.longitude"},
                "geocode.latitude": {"$toDouble": "$geocode.latitude"}
            }
        }
    ],
)

In [88]:
test = establishments.find_one()
print(type(test["geocode"]["latitude"]))
print(type(test["geocode"]["longitude"]))

<class 'float'>
<class 'float'>


Use `update_many` to convert `RatingValue` to integer numbers.

In [90]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [91]:
# Change the data type from String to Integer for RatingValue
establishments.update_many(
    {},
    [
        {
            "$set": {
                "RatingValue": {"$toInt": "$RatingValue"}
            }
        }
    ]
)

In [96]:
# Check that the coordinates and rating value are now numbers
test = establishments.find_one()
longitude = test["geocode"]["longitude"]
latitude = test["geocode"]["latitude"]
RatingValue = test["RatingValue"]

print(f'Longitude data type: {type(longitude)}')
print(f'Latitude data type: {type(latitude)}')
print(f'RatingValue data type: {type(RatingValue)}')

Longitude data type: <class 'float'>
Latitude data type: <class 'float'>
RatingValue data type: <class 'int'>
